In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import pingouin as pg
import plotly.graph_objects as go
from os.path import join as pjoin

sys.path.append("../..")
import circletrack_behavior as ctb
import plotting_functions as pf

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
## Settings
lin_path = '../../../MultiCon_Behavior/MultiCon_Pilot2/output/lin_behav/'
circle_path = '../../../MultiCon_Behavior/MultiCon_Pilot2/output/behav/'
fig_path = '../../../MultiCon_Behavior/MultiCon_Pilot2/intermediate_figures'
two_ports = ['mcp01', 'mcp02', 'mcp03', 'mcp05', 'mcp08', 'mcp10']
thresh_five = ['mcp01', 'mcp03', 'mcp06', 'mcp08', 'mcp09', 'mcp11']
two_color_plots = ['darkgrey', 'midnightblue']
four_color_plots = ['darkgreen', 'darkgrey', 'darkblue', 'blue']
group_colors_dict = {'thresh3_twoport': 'darkgrey', 'thresh3_oneport': 'darkgreen', 'thresh5_twoport': 'midnightblue', 'thresh5_oneport': 'darkblue'}
chance_color = 'darkgrey'
bin_size = 4 ## for binning across trials
error_colors = ['rgba(169,169,169,0.4)', 'rgba(25,25,112,0.4)'] ## rgba is the only way to get the error band transparent
opacity = 0.8 ## for binning across trials

### Analyze linear track data.

In [ ]:
## Linear track rewards
result_dict = {'mouse': [], 'day': [], 'group': [], 'rewards': []}
for mouse in os.listdir(lin_path):
    mouse_path = pjoin(lin_path, mouse)
    group = 'thresh3_twoport' if mouse in two_ports and mouse not in thresh_five else 'thresh5_twoport' if mouse in two_ports and mouse in thresh_five else 'thresh3_oneport' if mouse not in two_ports and mouse not in thresh_five else 'thresh5_oneport'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        result_dict['mouse'].append(mouse)
        result_dict['day'].append(idx+1)
        result_dict['group'].append(group)
        result_dict['rewards'].append(np.sum(behav['water']))
results_df = pd.DataFrame(result_dict)

In [ ]:
## Plot rewards across days in linear track
fig = pf.plot_behavior_across_days(results_df, x_var='day', y_var='rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=four_color_plots, avg_color='darkgrey', expert_line=False, chance=False,
                                   x_title='Day', y_title='Rewards', titles=['Linear Track'], height=500, width=600)
fig.show()
results_df.mixed_anova(dv='rewards', within='day', subject='mouse', between='group')
fig.write_image(pjoin(fig_path, 'linear_track_rewards.png'))

### Analyze circle track data.

In [12]:
circletrack_results = {'mouse': [], 'day': [], 'group': [], 'session': [], 'lick_accuracy': [], 'rewards': []}
for mouse in os.listdir(circle_path):
    mouse_path = pjoin(circle_path, mouse)
    group = 'thresh3_twoport' if mouse in two_ports and mouse not in thresh_five else 'thresh5_twoport' if mouse in two_ports and mouse in thresh_five else 'thresh3_oneport' if mouse not in two_ports and mouse not in thresh_five else 'thresh5_oneport'
    if (group == 'thresh3_oneport') | (group == 'thresh5_oneport'):
          pass
    else:
        for idx, session in enumerate(os.listdir(mouse_path)):
                behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
                behav_data = behav[~behav['probe']]
                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                session_pc = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, by_trials=False)
                circletrack_results['mouse'].append(mouse)
                circletrack_results['day'].append(idx+1)
                circletrack_results['group'].append(group)
                circletrack_results['session'].append(np.unique(behav['session'])[0])
                circletrack_results['lick_accuracy'].append(session_pc)
                circletrack_results['rewards'].append(np.sum(behav_data['water']))
ct_df = pd.DataFrame(circletrack_results)

In [13]:
## Plot lick accuracy across days
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy', groupby_var=['day', 'group'], plot_transitions=[6.5, 12.5],
                                   marker_color=two_color_plots, avg_color='darkgrey', expert_line=True, chance=True,
                                   transition_color=['darkgrey', 'darkgrey'],
                                   x_title='Day', y_title='Lick Accuracy', titles=['Circle Track'], height=500, width=600)
fig.update_yaxes(range=[0, 100], dtick=25)
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy.png'))
ct_df.mixed_anova(dv='lick_accuracy', within='day', subject='mouse', between='group')

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,1.900719,1,4,1.900719,0.004569,9.493545e-01,0.001141,NaN
1,day,13856.117561,13,52,1065.855197,12.987790,4.544595e-12,0.764537,0.257918
2,Interaction,586.870320,13,52,45.143871,0.550093,8.811793e-01,0.120897,NaN


In [14]:
## Plot rewards across days
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='rewards', groupby_var=['day', 'group'], plot_transitions=[6.5, 12.5],
                                   marker_color=two_color_plots, avg_color='darkgrey', expert_line=False, chance=False,
                                   transition_color=['darkgrey', 'darkgrey'],
                                   x_title='Day', y_title='Rewards', titles=['Circle Track'], height=500, width=600)
fig.show()
fig.write_image(pjoin(fig_path, 'total_rewards.png'))
ct_df.mixed_anova(dv='rewards', within='day', subject='mouse', between='group')

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,7298.678571,1,4,7298.678571,2.345226,2.004239e-01,0.369605,NaN
1,day,27091.488095,13,52,2083.960623,8.295368,1.059819e-08,0.674674,0.218154
2,Interaction,5268.154762,13,52,405.242674,1.613100,1.115461e-01,0.287381,NaN


### Signal detection metrics across all mice.

In [4]:
metrics = pd.DataFrame()
for mouse in os.listdir(circle_path):
    mouse_path = pjoin(circle_path, mouse)
    group = 'thresh3_twoport' if mouse in two_ports and mouse not in thresh_five else 'thresh5_twoport' if mouse in two_ports and mouse in thresh_five else 'thresh3_oneport' if mouse not in two_ports and mouse not in thresh_five else 'thresh5_oneport'
    if (group == 'thresh3_oneport') | (group == 'thresh5_oneport'):
          pass
    else:
        for idx, session in enumerate(os.listdir(mouse_path)):
            behav_data = pd.read_feather(pjoin(mouse_path, f'{session}'))
            reward_one, reward_two = np.unique(behav_data['reward_one'])[0], np.unique(behav_data['reward_two'])[0]
            signal = pd.DataFrame(ctb.dprime_metrics(behav_data, mouse, day=idx+1, reward_ports=[reward_one, reward_two], forward_reverse='forward'))
            signal['group'] = group
            metrics = pd.concat([metrics, signal], ignore_index=True)

In [8]:
## dprime across contexts
fig = pf.plot_behavior_across_days(data=metrics, x_var='day', y_var='dprime', groupby_var=['day', 'group'], plot_transitions=[6.5, 12.5],
                                   transition_color=[chance_color, chance_color], width=600,
                                   x_title='Day', y_title="d'", marker_color=two_color_plots, expert_line=False, chance=False, plot_datapoints=False)
fig.show()
fig.write_image(pjoin(fig_path, 'dprime_across_days.png'))

In [10]:
## Correct rejection rate across contexts
fig = pf.plot_behavior_across_days(data=metrics, x_var='day', y_var='CR', groupby_var=['day', 'group'], plot_transitions=[6.5, 12.5],
                                   transition_color=[chance_color, chance_color], width=600,
                                   x_title='Day', y_title='Correct Rejection Rate', marker_color=two_color_plots, expert_line=False, chance=False, plot_datapoints=False)
fig.add_hline(y=1, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.show()
fig.write_image(pjoin(fig_path, 'CR_across_days.png'))

In [11]:
## Hit rate rate across contexts
fig = pf.plot_behavior_across_days(data=metrics, x_var='day', y_var='hits', groupby_var=['day', 'group'], plot_transitions=[6.5, 12.5],
                                   transition_color=[chance_color, chance_color], width=600,
                                   x_title='Day', y_title='Hit Rate', marker_color=two_color_plots, expert_line=False, chance=False, plot_datapoints=False)
fig.add_hline(y=1, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.show()
fig.write_image(pjoin(fig_path, 'HR_across_days.png'))

### Probe accuracy during the first and last day in a context.

In [44]:
lick_dict_probe = {'mouse': [], 'day': [], 'group': [], 'session': [], 'num_licks': [], 'probe_pc': [], 'session_pc': []}
for mouse in os.listdir(circle_path):
    mouse_path = pjoin(circle_path, mouse)
    group = 'thresh3_twoport' if mouse in two_ports and mouse not in thresh_five else 'thresh5_twoport' if mouse in two_ports and mouse in thresh_five else 'thresh3_oneport' if mouse not in two_ports and mouse not in thresh_five else 'thresh5_oneport'
    if (group == 'thresh3_oneport') | (group == 'thresh5_oneport'):
          pass
    else:
        for idx, session in enumerate(os.listdir(mouse_path)):
            behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
            if any(behav['probe']):
                behav_probe = behav.loc[behav['probe']]
                behav_no_probe = behav[~behav['probe']]
                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                ## Percent correct licking
                pc = ctb.lick_accuracy(behav_probe, port_one=reward_one, port_two=reward_two, by_trials=False)
                session_pc = ctb.lick_accuracy(behav_no_probe, port_one=reward_one, port_two=reward_two, by_trials=False)
                lick_dict_probe['mouse'].append(mouse)
                lick_dict_probe['day'].append(idx+1)
                lick_dict_probe['group'].append(group)
                lick_dict_probe['session'].append(np.unique(behav['session'])[0])
                lick_dict_probe['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
                lick_dict_probe['probe_pc'].append(pc)
                lick_dict_probe['session_pc'].append(session_pc)
            else:
                pass
## Convert to dataframe
probe_df = pd.DataFrame(lick_dict_probe)
avg_probe = probe_df.groupby(['group', 'day'], as_index=False).agg({'probe_pc': ['mean', 'sem']})

In [46]:
## Plot average probe performance
fig = pf.custom_graph_template(x_title='Day', y_title='Probe Accuracy', width=600)
for group in np.unique(avg_probe['group']):
    group_data = avg_probe[avg_probe['group'] == group]
    fig.add_trace(go.Scatter(x=group_data['day'], y=group_data['probe_pc']['mean'], name=group, mode='lines+markers',
                            line_color=group_colors_dict[group], error_y=dict(type='data', array=group_data['probe_pc']['sem'])))
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_vline(x=6.5, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_vline(x=12.5, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.show() 
fig.write_image(pjoin(fig_path, 'probe_accuracy.png'))

### Plot lick accuracy, dprime, correct rejection rate across trials for all days.

In [29]:
## Bin metrics
variable_of_interest = 'dprime'
only_min_trials = False
## Bin data
binned_metrics = ctb.aggregate_metrics(metrics, bin_size=bin_size, variable_of_interest=variable_of_interest)
avg_binned = binned_metrics.groupby(['group', 'day', 'binned_trial'], as_index=False).agg({variable_of_interest: ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(avg_binned['day']))+1)])
for idx, day in enumerate(np.unique(avg_binned['day'])):
    plot_data = avg_binned[avg_binned['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in np.unique(plot_data['group']):
            trial_lengths.append(plot_data['binned_trial'][plot_data['group'] == group].values[-1])
            min_trials = np.min(trial_lengths)
    if idx < 6:
        row, col = 1, idx + 1
    elif (idx >= 6) & (idx < 12):
        row, col = 2, idx - 5
    else:
        row, col = 3, idx - 11

    for ind, group in enumerate(np.unique(plot_data['group'])):
            if only_min_trials:
                group_data = plot_data[(plot_data['group'] == group) & (plot_data['binned_trial'] <= min_trials)]
                upper = group_data[variable_of_interest]['mean'] + group_data[variable_of_interest]['sem']
                lower = group_data[variable_of_interest]['mean'] - group_data[variable_of_interest]['sem']
            else:
                group_data = plot_data[(plot_data['group'] == group)]
                upper = group_data[variable_of_interest]['mean'] + group_data[variable_of_interest]['sem']
                lower = group_data[variable_of_interest]['mean'] - group_data[variable_of_interest]['sem']
            x_data = np.arange(1, len(group_data['binned_trial'])+1) * bin_size
            fig.add_trace(go.Scatter(x=x_data, y=group_data[variable_of_interest]['mean'], mode='lines', opacity=opacity,
                                    line_color=group_colors_dict[group], showlegend=False,
                                    legendgroup=group, name=group), row=row, col=col)
            fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                    marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                    legendgroup=group), row=row, col=col)
            fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                    line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                    showlegend=False, legendgroup=group), row=row, col=col)
if variable_of_interest != 'dprime':
    fig.add_hline(y=1, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title="d'", col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
fig.write_image(pjoin(fig_path, 'dprime_across_trials_across_days.png'))

In [31]:
## Bin metrics
variable_of_interest = 'CR'
only_min_trials = False
## Bin data
binned_metrics = ctb.aggregate_metrics(metrics, bin_size=bin_size, variable_of_interest=variable_of_interest)
avg_binned = binned_metrics.groupby(['group', 'day', 'binned_trial'], as_index=False).agg({variable_of_interest: ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(avg_binned['day']))+1)])
for idx, day in enumerate(np.unique(avg_binned['day'])):
    plot_data = avg_binned[avg_binned['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in np.unique(plot_data['group']):
            trial_lengths.append(plot_data['binned_trial'][plot_data['group'] == group].values[-1])
            min_trials = np.min(trial_lengths)
    if idx < 6:
        row, col = 1, idx + 1
    elif (idx >= 6) & (idx < 12):
        row, col = 2, idx - 5
    else:
        row, col = 3, idx - 11

    for ind, group in enumerate(np.unique(plot_data['group'])):
            if only_min_trials:
                group_data = plot_data[(plot_data['group'] == group) & (plot_data['binned_trial'] <= min_trials)]
                upper = group_data[variable_of_interest]['mean'] + group_data[variable_of_interest]['sem']
                lower = group_data[variable_of_interest]['mean'] - group_data[variable_of_interest]['sem']
            else:
                group_data = plot_data[(plot_data['group'] == group)]
                upper = group_data[variable_of_interest]['mean'] + group_data[variable_of_interest]['sem']
                lower = group_data[variable_of_interest]['mean'] - group_data[variable_of_interest]['sem']
            x_data = np.arange(1, len(group_data['binned_trial'])+1) * bin_size
            fig.add_trace(go.Scatter(x=x_data, y=group_data[variable_of_interest]['mean'], mode='lines', opacity=opacity,
                                    line_color=group_colors_dict[group], showlegend=False,
                                    legendgroup=group, name=group), row=row, col=col)
            fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                    marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                    legendgroup=group), row=row, col=col)
            fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                    line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                    showlegend=False, legendgroup=group), row=row, col=col)
if variable_of_interest != 'dprime':
    fig.add_hline(y=1, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Correct Rejection', col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
fig.write_image(pjoin(fig_path, 'CR_across_trials_across_days.png'))

In [18]:
## Calculate for lick accuracy
data = {'mouse': [], 'day': [], 'group': [], 'trial_acc': []}
for mouse in os.listdir(circle_path):
    mouse_path = pjoin(circle_path, mouse)
    group = 'thresh3_twoport' if mouse in two_ports and mouse not in thresh_five else 'thresh5_twoport' if mouse in two_ports and mouse in thresh_five else 'thresh3_oneport' if mouse not in two_ports and mouse not in thresh_five else 'thresh5_oneport'
    if (group == 'thresh3_oneport') | (group == 'thresh5_oneport'):
          pass
    else:
        for idx, session in enumerate(os.listdir(mouse_path)):
            behav_data = pd.read_feather(pjoin(mouse_path, f'{session}'))
            reward_one, reward_two = np.unique(behav_data['reward_one'])[0], np.unique(behav_data['reward_two'])[0]
            pc = ctb.lick_accuracy(behav_data, reward_one, reward_two, by_trials=True)
            binned_pc = ctb.bin_data(pc, bin_size=bin_size)
            x_data = np.arange(1, len(binned_pc) + 1) * bin_size
            data['mouse'].append(mouse)
            data['day'].append(idx + 1)
            data['group'].append(group)
            data['trial_acc'].append(binned_pc)
trial_acc = pd.DataFrame(data)
group_data = {'thresh3_twoport': [], 'thresh5_twoport': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['thresh3_twoport'].append(np.nanmean(trial_acc['trial_acc'][(trial_acc['group'] == 'thresh3_twoport') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['thresh5_twoport'].append(np.nanmean(trial_acc['trial_acc'][(trial_acc['group'] == 'thresh5_twoport') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['thresh3_twoport'].append(np.std(trial_acc['trial_acc'][(trial_acc['group'] == 'thresh3_twoport') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc'][(trial_acc['group'] == 'thresh3_twoport') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['thresh5_twoport'].append(np.std(trial_acc['trial_acc'][(trial_acc['group'] == 'thresh5_twoport') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc'][(trial_acc['group'] == 'thresh5_twoport') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
group_df = pd.DataFrame(group_data)

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Behavior/MultiCon_Pilot2/../../circletrack_behavior.py:1068: RuntimeWarning:

invalid value encountered in scalar divide

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Behavior/MultiCon_Pilot2/../../circletrack_behavior.py:1068: RuntimeWarning:

invalid value encountered in scalar divide

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Behavior/MultiCon_Pilot2/../../circletrack_behavior.py:1068: RuntimeWarning:

invalid value encountered in scalar divide

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Behavior/MultiCon_Pilot2/../../circletrack_behavior.py:1068: RuntimeWarning:

invalid value encountered in scalar divide

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Behavior/MultiCon_Pilot2/../../circletrack_behavior.py:1068: RuntimeWarning:

invalid value encountered in scalar divide

/media/cai

In [25]:
## For lick accuracy
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(group_df['day']))+1)])
for idx, day in enumerate(np.unique(group_df['day'])):
    plot_data = group_df[group_df['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
if only_min_trials:
    fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_across_groups.png'))
else:
    fig.write_image(pjoin(fig_path, 'lick_acc_across_groups.png'))

### Look at lick thresh 3 vs lick thresh 5.

In [3]:
circletrack_results = {'mouse': [], 'day': [], 'group': [], 'session': [], 'lick_accuracy': [], 'rewards': []}
for mouse in os.listdir(circle_path):
    mouse_path = pjoin(circle_path, mouse)
    group = 'thresh3_twoport' if mouse in two_ports and mouse not in thresh_five else 'thresh5_twoport' if mouse in two_ports and mouse in thresh_five else 'thresh3_oneport' if mouse not in two_ports and mouse not in thresh_five else 'thresh5_oneport'
    if (group == 'thresh3_oneport') | (group == 'thresh5_oneport'):
          pass
    else:
        for idx, session in enumerate(os.listdir(mouse_path)):
                behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
                behav_data = behav[~behav['probe']]
                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                if group == 'thresh3_twoport':
                    thresh = 3
                elif group == 'thresh5_twoport':
                    thresh = 5
                session_pc = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=5, by_trials=False)
                circletrack_results['mouse'].append(mouse)
                circletrack_results['day'].append(idx+1)
                circletrack_results['group'].append(group)
                circletrack_results['session'].append(np.unique(behav['session'])[0])
                circletrack_results['lick_accuracy'].append(session_pc)
                circletrack_results['rewards'].append(np.sum(behav_data['water']))
ct_df = pd.DataFrame(circletrack_results)

In [6]:
## Plot lick accuracy across days for thresh3 or thresh5
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy', groupby_var=['day', 'group'], plot_transitions=[6.5, 12.5],
                                   marker_color=two_color_plots, avg_color='darkgrey', expert_line=True, chance=True,
                                   transition_color=['darkgrey', 'darkgrey'], plot_datapoints=False,
                                   x_title='Day', y_title='Lick Accuracy', titles=['Circle Track'], height=500, width=600)
fig.update_yaxes(range=[0, 100], dtick=25)
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy.png'))
ct_df.mixed_anova(dv='lick_accuracy', within='day', subject='mouse', between='group')

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,3124.314407,1,4,3124.314407,1.609247,2.733987e-01,0.286892,NaN
1,day,19919.428240,13,52,1532.263711,13.694896,1.677903e-12,0.773946,0.22063
2,Interaction,1841.749515,13,52,141.673040,1.266230,2.631148e-01,0.240443,NaN
